In [42]:
import warnings
warnings.filterwarnings('ignore')

import os
from glob import glob
import pickle

import numpy as np
import pandas as pd

#TEXT PROCESSING
import nltk
import re
import codecs
import unidecode
#pip install unidecode
import mpld3
# pip install mpld3
import stop_words
# pip install stop-words
from nltk import SnowballStemmer, pos_tag, word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
import string,re

#GENSIM MODELS
import gensim
import logging
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import pyemd

#SKLEARN
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import *
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import *
from sklearn.metrics import pairwise


### fonctions de pretraitement

In [2]:
regex = re.compile('[%s]' % '(\\n)*(\\x0c)*')

def del_line_feed(s):  
    """Delete \n in the text"""
    return regex.sub(' ', s)

In [3]:
#le maintient de la ponctuation pertube le stop words, apostrophe gérée dans text_treatment
regex = re.compile('[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_{|}~')) 

def del_punct(s):  
    """Delete punctuation in the text"""
    return regex.sub('', s)

In [4]:
def _calculate_languages_ratios(text):
    """
    Calculate probability of given text to be written in several languages and
    return a dictionary that looks like {'french': 2, 'spanish': 4, 'english': 0}
    """

    languages_ratios = {}

    '''
    nltk.wordpunct_tokenize() splits all punctuations into separate tokens
    
    >>> wordpunct_tokenize("That's thirty minutes away. I'll be there in ten.")
    ['That', "'", 's', 'thirty', 'minutes', 'away', '.', 'I', "'", 'll', 'be', 'there', 'in', 'ten', '.']
    '''

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens] #from text get list of word in minuscule

    
    for language in stopwords.fileids(): # pour chaque langue
        stopwords_set = set(stopwords.words(language)) #je mets les stop words du langage dans un set
        words_set = set(words) #je mets les mots de mon texte dans un set
        #je prends l'intersection entre les mots de mon texte et les mots du stopwords dans le langage donné
        common_elements = words_set & stopwords_set
        
        #je compute mon score comme le nombre d'éléments en communs dictionnaire [langage : score]
        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios

In [5]:
def get_cv_langue(liste_cv, language) :
    """Return resume witten in the specified language in parameter"""
    liste_2 = []
    for cv in liste_cv:
        if max(_calculate_languages_ratios(cv),key =_calculate_languages_ratios(cv).get)=='french':
            liste_2.append(cv)
    return liste_2

In [6]:
nltk.download('stopwords')
stopwords.fileids()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mehdiregina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish',
 'turkish']

In [7]:
def text_treatment (text):
    text = text.lower()
    text = text.replace("\x00", '').replace("\x01", '').replace("\x02", '').replace("\x03", '') \
    .replace("\x04", '').replace("\x05", '').replace("\x06", '').replace("\x07", '').replace("\x08", '') \
    .replace("\x0e", '').replace("\x11", '').replace("\x12", '').replace("\x10", '').replace("\x19", '') \
    .replace("\x1b", '').replace("\x14", '').replace("\x15", '').replace('/', ' ').replace('=', ' ').replace("〓", "") \
    .replace("»", "").replace("«", "").replace("¬", "").replace('`', '').replace (" -", " ").replace("•", "")\
    .replace("l'", "").replace("l’", "").replace("l´", "").replace("d’", "").replace("d'", "").replace("d´","")\
    .replace("j’", "").replace("j'", "").replace("j´","").replace("n’", "").replace("n'", "").replace("n´","")\
    .replace("”", "").replace("~", "").replace("§", "").replace("¨", "").replace("©", "").replace("›", "")\
    .replace("₋", "").replace("→", "").replace("⇨", "").replace("∎", "").replace("√", "").replace("□", "")\
    .replace("*", "").replace("&", "").replace("►", "").replace("◊", "").replace("☞", "").replace("#", "")\
    .replace("%", "").replace("❖", "").replace("➠", "").replace("➢", "").replace("", "").replace("✓", "") \
    .replace("√", "").replace("✔", "").replace("♦", "").replace("◦", "").replace("●", "").replace("▫", "")\
    .replace("▪", "").replace("…", "").replace("þ", "").replace("®", "").replace('', '').replace("...", "")
    text = unidecode.unidecode(text) # remove accent
    return text

In [8]:
#generate stopwords
stop_words_py = set(stop_words.get_stop_words('french'))

# attention certains stop words pourraient être utiles par la suite
stopwords_set_manuel = set(["an", "ans", 'les', 'moins', 'd\'un','janvier', 'fevrier', 'février', 'mars', 'avril', \
                 'mai', 'juin', 'juillet', 'aout', 'août', 'septembre', 'octobre', 'novembre', 'décembre', \
                  'decembre', 'moins', 'mise', 'universit\xc3\xa9', 'université', 'universite', 'ion','sage', \
                  'o', 'rac', 'vers', 'via', 'p\xc3\xa9rim\xc3\xa8tre', 'périmètre','et','paris','x',"\x00",\
                          "\x01","\x02", "\x03","\x04","\x05","\x06","\x07","\x08","\x09","\x0e","\x0e","\x11",\
                           "\x12","\x13","\x14","\x15","\x16","\x17","\x18","\x19","transport","puis","lieu",\
                           "adresse","entre",'dun','dune','chez','boulognebillancourt','bt','etc','recrutement','main',\
                           'and', 'paie','paiement','environ','place','france','paris','mois','mobile','mobiles',\
                           'nanterre','source','sources','concerne','concernant','of','non','notes','rh','minimum',\
                           'maximum','bac','site','sites','actuellement','telephone','telephonique','telephoniques','ca','demenager',\
                           'demenagement','participer','participation','lycee','baccalaureat','lien','liens','in',\
                           'indeed','email','indeedcomrd7e8913ed00d0384','aujourhui','afin','toujours','enterprise',\
                           "guide","10g","11g","9i",'ad','v10','v2','v3','v5','v6','v8','v9','talan','talansolutions',\
                           "aide","ainsi",'aix','aupres','autour','autres','b','bonne','campagnes','cas','chaine',\
                           'choix','coherence','departement','differentes','differents','divers','fin','for','grandes',\
                           'i','ii','jour','lies','lors','lu','mettre','necessaires','national','nationale','nouvelle',\
                           'nouvelles','parle','partir','partie','permettant','permettte','plusieurs','reel','selon',\
                           'temps','toutes','v'])
stop_words_main = stop_words_py | stopwords_set_manuel
stop_words_main = list(stop_words_main)
print("taille stop words liste : ", len(stop_words_main))

#voir si utile
def remove_stopwords(text,stopwords_list):
    text_temp = " ".join(text.split())+" "
    for word in stopwords_list:
        text_temp = text_temp.replace(" "+word+" ", " ")
    return text_temp

taille stop words liste :  413


In [62]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #print(tokens)
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

CV TALAN

In [11]:
liste_cv_talan = []
path = '../Data_talan/txt/'
filenames = sorted(glob(os.path.join(path,"*.txt")))
for file in filenames:
    liste_cv_talan.append(open(file).read())
len(filenames)

218

In [13]:
#On supprime les caractères étranges et accents
liste_cv_talan_no_punc = [del_punct(text) for text in liste_cv_talan]
liste_cv_talan_fr = get_cv_langue(liste_cv_talan_no_punc,'french')
liste_cv_talan_treated = [text_treatment(text) for text in liste_cv_talan_fr]
liste_cv_talan_no_stop = [remove_stopwords(text,stop_words_main) for text in liste_cv_talan_treated]
liste_cv_talan_clean = [re.sub('[0-9 ]+', ' ', text) for text in liste_cv_talan_no_stop]

In [14]:
liste_cv_talan_clean[10]

'bena ingenieur affaires domaines competences communicant ecoute capacite comprendre enjeux attentes prise charge opportunites detection closing direct travers reseau partenaires sens initiative sens management equipes pilotage propositions commerciales negociation contrats animation parc clients grands comptes detection projets filiales pilotage ressources internes consultants avantvente experts techniques juristes garant satisfaction client suivi facturation oeuvre projets complexes forte experience professionnelle sept ingenieur commercial - eme prospection clients qualification opportunites produits x perimetre donne strategie developpement reponse appels offres construction strategie specifique client redaction propositions commerciales licences services presentation offre globale licences services pilotage equipes reponses vente services projet complexe forte construction offre tma consolidation comptes vente complementaire services licences reunions lancement chef projet seniora

Word2Vec Implementation

Quid ->
    - Supprimer une partie des mots via tf-idf avant le word2vec ?
    - Considérer tout ou une sous partie du cv ?

In [15]:
def del_word_freq(max_idf,min_idf,list_texts,tokenisation=True):
    
    tf_vect = TfidfVectorizer(stop_words=stop_words_main,max_df=max_idf,min_df=min_idf,\
                           preprocessor=text_treatment,tokenizer=tokenize_only)
    bow_idf_indeed = tf_vect.fit_transform(list_texts)
    vocab_liste_idf = tf_vect.get_feature_names()
    print(len(vocab_liste_idf))
    
    liste_texte_new =[]
    prop = 0
    for text in list_texts:
        liste_mot = [mot for mot in text.split(' ') if mot in vocab_liste_idf]
        liste_texte_new.append(' '.join(liste_mot))
        prop += len(liste_mot)/len(text.split(' '))
    
    if tokenisation:
        liste_texte_new = [tokenize_only(text) for text in liste_texte_new]
        
    return liste_texte_new, (prop/len(list_texts)), vocab_liste_idf

In [144]:
#suppression au niveau des mots via le tf-idf
liste_tokens_2, prop, vocab_liste_idf = del_word_freq(0.8,0.1,liste_cv_talan_clean)

526


In [145]:
#tokenization des textes pas de suppression au niveau des mots
liste_tokens = [tokenize_only(text) for text in liste_cv_talan_clean]

In [146]:
vocab_liste_idf

['acces',
 'access',
 'accompagne',
 'accompagnement',
 'achat',
 'achats',
 'acteurs',
 'actions',
 'activite',
 'activites',
 'adaptation',
 'administration',
 'agile',
 'alimentation',
 'amc',
 'amelioration',
 'amoa',
 'analyse',
 'analyser',
 'analyses',
 'analyst',
 'analyste',
 'analytics',
 'anglais',
 'animation',
 'annee',
 'annees',
 'anomalies',
 'appels',
 'applicatif',
 'applicatifs',
 'application',
 'applications',
 'applicative',
 'arabe',
 'architecture',
 'aspects',
 'assistance',
 'assurance',
 'ateliers',
 'audit',
 'automatique',
 'automatisation',
 'avancement',
 'banque',
 'base',
 'bases',
 'basic',
 'besoin',
 'besoins',
 'bi',
 'big',
 'bilingue',
 'bnp',
 'bo',
 'bord',
 'budget',
 'budgetaire',
 'business',
 'c',
 'cabinet',
 'cadrage',
 'cadre',
 'cahier',
 'cahiers',
 'calcul',
 'cartographie',
 'center',
 'centre',
 'centres',
 'certification',
 'certifications',
 'changement',
 'charge',
 'charges',
 'chef',
 'chiffrage',
 'cible',
 'client',
 'clients'

In [131]:
liste_tokens[0]

['secteurs',
 'telecom',
 'media',
 'activites',
 'metier',
 'avantvente',
 'developpement',
 'integration',
 'validation',
 'competences',
 'fonctionnelles',
 'crm',
 'elaboration',
 'reponses',
 'appels',
 'offres',
 'conception',
 'processus',
 'metiers',
 'redaction',
 'specifications',
 'fonctionnelles',
 'preparations',
 'maquettes',
 'competences',
 'techniques',
 'microsoft',
 'dynamics',
 'crm',
 'salesforce',
 'crm',
 'architecture',
 'plan',
 'migration',
 'volumetrie',
 'importante',
 'service',
 'director',
 'solution',
 'qos',
 'oblicore',
 'guarantee',
 'solution',
 'slm',
 'developpement',
 'j',
 'ee',
 'ejb',
 'jms',
 'webservice',
 'api',
 'javascriptc',
 'base',
 'donnees',
 'sql',
 'server',
 'oracle',
 'postgresql',
 'mysql',
 'methodologie',
 'agile',
 'scrum',
 'extreem',
 'programing',
 'sure',
 'step',
 'uml',
 'atos',
 'bull',
 'crm',
 'dynamics',
 'salesforce',
 'crm',
 'dynamics',
 'avantvente',
 'crm',
 'elaboration',
 'reponses',
 'appels',
 'offres',
 'co

In [161]:
import warnings
warnings.filterwarnings('ignore')

# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
sentences = liste_tokens_2
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences ,size=90)

2018-04-03 15:13:47,966 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-04-03 15:13:47,969 : INFO : collecting all words and their counts
2018-04-03 15:13:47,971 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-03 15:13:47,998 : INFO : collected 526 word types from a corpus of 62406 raw words and 215 sentences
2018-04-03 15:13:47,999 : INFO : Loading a fresh vocabulary
2018-04-03 15:13:48,004 : INFO : min_count=5 retains 526 unique words (100% of original 526, drops 0)
2018-04-03 15:13:48,006 : INFO : min_count=5 leaves 62406 word corpus (100% of original 62406, drops 0)
2018-04-03 15:13:48,012 : INFO : deleting the raw counts dictionary of 526 items
2018-04-03 15:13:48,014 : INFO : sample=0.001 downsamples 110 most-common words
2018-04-03 15:13:48,016 : INFO : downsampling leaves estimated 51045 word corpus (81.8% of prior 62406)
2018-04-03 15:13:48,020 : INFO : estimated required memory for 526 words and 90 dim

In [160]:
model.similar_by_word("modelisation")

2018-04-03 15:10:41,419 : INFO : precomputing L2-norms of word weight vectors


[('optimisation', 0.9976205229759216),
 ('traitements', 0.9962619543075562),
 ('livraison', 0.9957306385040283),
 ('jeux', 0.995457112789154),
 ('automatisation', 0.9951083660125732),
 ('interface', 0.9948158264160156),
 ('verification', 0.9946675300598145),
 ('fonctionnalites', 0.9945410490036011),
 ('integration', 0.994412362575531),
 ('datawarehouse', 0.9942609667778015)]

### Test en faisant un stemming au préalable afin de reduire le nombre de mots 

In [67]:
#tokenization des textes pas de suppression au niveau des mots
#SnowballStemmer 
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french")

liste_tokens_stem = [tokenize_and_stem(text) for text in liste_cv_talan_clean]

import warnings
warnings.filterwarnings('ignore')

# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
sentences = liste_tokens_stem
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences ,size=50)

2018-04-03 14:40:51,780 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-04-03 14:40:51,782 : INFO : collecting all words and their counts
2018-04-03 14:40:51,784 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-03 14:40:51,831 : INFO : collected 7992 word types from a corpus of 111420 raw words and 215 sentences
2018-04-03 14:40:51,833 : INFO : Loading a fresh vocabulary
2018-04-03 14:40:51,863 : INFO : min_count=5 retains 2163 unique words (27% of original 7992, drops 5829)
2018-04-03 14:40:51,866 : INFO : min_count=5 leaves 101684 word corpus (91% of original 111420, drops 9736)
2018-04-03 14:40:51,886 : INFO : deleting the raw counts dictionary of 7992 items
2018-04-03 14:40:51,889 : INFO : sample=0.001 downsamples 73 most-common words
2018-04-03 14:40:51,891 : INFO : downsampling leaves estimated 85131 word corpus (83.7% of prior 101684)
2018-04-03 14:40:51,906 : INFO : estimated required memory for 2163 word

In [68]:
model.similar_by_word("d")

2018-04-03 14:40:52,537 : INFO : precomputing L2-norms of word weight vectors


[('api', 0.9993228912353516),
 ('team', 0.999142587184906),
 ('ser', 0.999042809009552),
 ('eai', 0.9987051486968994),
 ('salesforc', 0.9986593723297119),
 ('pag', 0.998621940612793),
 ('so', 0.9986141324043274),
 ('introduct', 0.9986059665679932),
 ('fixed', 0.9985703229904175),
 ('map', 0.9985432624816895)]

Cette méthode ne prend pas en compte le tf idf associé à chaque document ?

In [94]:
model.init_sims(replace=True)

model.wmdistance(liste_cv_talan_clean[0],liste_cv_talan_clean[50])

2018-04-03 14:54:14,406 : INFO : precomputing L2-norms of word weight vectors
2018-04-03 14:54:14,428 : INFO : Removed 8894 and 9777 OOV words from document 1 and 2 (respectively).
2018-04-03 14:54:14,429 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-04-03 14:54:14,432 : INFO : built Dictionary(3 unique tokens: ['c', 'g', 'r']) from 2 documents (total 2339 corpus positions)


0.0077250125866837665

IMPLEMENTATION WORDMOVER

Earth mover distance prend en entrée deux vecteurs de même dimension, et la matrice de distance entre les features et retourne la earth mover distance.

In [162]:
print("La taille du vocabulaire est : "+str(len(model.wv.vocab)))

La taille du vocabulaire est : 526


Les attributs ci-dessous permettent d'obtenir la matrice des mots exprimée dans l'encodage

In [96]:
word_vec_matrix = model.wv.syn0

Attention on constate que l'odonnancement des mots n'est pas le même que dans un tf-idf, il y a un travail de ré-ordonnancement à faire pour rester cohérent avec l'odre du tf idf

In [97]:
vocab_list_word_vec = model.wv.index2word

In [98]:
vocab_list_word_vec

['oracle',
 'data',
 'analyse',
 'sql',
 'conception',
 'specifications',
 'consultant',
 'realisation',
 'recette',
 'bi',
 'sas',
 'projets',
 'application',
 'production',
 'techniques',
 'migration',
 'pilotage',
 'utilisateurs',
 'creation',
 'tests',
 'reporting',
 'metier',
 'besoins',
 'integration',
 'clients',
 'client',
 'server',
 'fonctionnelles',
 'assistance',
 'systeme',
 'equipe',
 'base',
 'definition',
 'business',
 'solution',
 'support',
 'etude',
 'flux',
 'sap',
 'expertise',
 'modelisation',
 'decisionnel',
 'r',
 'cognos',
 'applications',
 'fonctionnelle',
 'outil',
 'talend',
 'informatique',
 'processus',
 'windows',
 'information',
 'maintenance',
 'ingenieur',
 'sein',
 'animation',
 'oeuvre',
 'rapports',
 'mission',
 'bo',
 'web',
 'alimentation',
 'architecture',
 'ibm',
 'management',
 'plateforme',
 'chef',
 'validation',
 'metiers',
 'optimisation',
 'etudes',
 'systemes',
 'responsable',
 'etl',
 'services',
 'activite',
 'service',
 'solutions',
 '

In [99]:
#compute euclidean distance between word in the encoded space
distance_matrix = pairwise.pairwise_distances(word_vec_matrix,metric='euclidean')
distance_matrix = distance_matrix.astype("float64")

In [100]:
#TF IDF BOW Representation
tf_vect = TfidfVectorizer(stop_words=stop_words_main,max_df=0.8,min_df=0.1,\
                           preprocessor=text_treatment,tokenizer=tokenize_only)
bow_idf = tf_vect.fit_transform(liste_cv_talan_clean)
buffer = pd.DataFrame(data=bow_idf.toarray())
buffer.drop_duplicates(inplace=True)
bow_idf = buffer.values
bow_idf = shuffle(bow_idf)
print(bow_idf.shape)

vocab_liste_idf = tf_vect.get_feature_names()

(214, 526)


214 CVs contre 218 précédemment il y a 4 CVs non dupliqués

In [101]:
vocab_liste_idf

['acces',
 'access',
 'accompagne',
 'accompagnement',
 'achat',
 'achats',
 'acteurs',
 'actions',
 'activite',
 'activites',
 'adaptation',
 'administration',
 'agile',
 'alimentation',
 'amc',
 'amelioration',
 'amoa',
 'analyse',
 'analyser',
 'analyses',
 'analyst',
 'analyste',
 'analytics',
 'anglais',
 'animation',
 'annee',
 'annees',
 'anomalies',
 'appels',
 'applicatif',
 'applicatifs',
 'application',
 'applications',
 'applicative',
 'arabe',
 'architecture',
 'aspects',
 'assistance',
 'assurance',
 'ateliers',
 'audit',
 'automatique',
 'automatisation',
 'avancement',
 'banque',
 'base',
 'bases',
 'basic',
 'besoin',
 'besoins',
 'bi',
 'big',
 'bilingue',
 'bnp',
 'bo',
 'bord',
 'budget',
 'budgetaire',
 'business',
 'c',
 'cabinet',
 'cadrage',
 'cadre',
 'cahier',
 'cahiers',
 'calcul',
 'cartographie',
 'center',
 'centre',
 'centres',
 'certification',
 'certifications',
 'changement',
 'charge',
 'charges',
 'chef',
 'chiffrage',
 'cible',
 'client',
 'clients'

In [211]:
def order_tfidf_matrix(tfidf_matrix,ex_order,new_order):
    tfidf_matrix = tfidf_matrix.toarray()
    print(tfidf_matrix.shape)
    buffer = np.zeros(tfidf_matrix.shape,dtype=np.float64)
    dict_ex_order = dict(zip(ex_order,range(len(ex_order))))
    for i,elt in enumerate(new_order):
        idx_elt = dict_ex_order[elt]
        buffer[:,i] = tfidf_matrix[:,idx_elt]
    return buffer

In [212]:
bow_idf_new_order = order_tfidf_matrix(bow_idf,vocab_liste_idf,vocab_list_word_vec)

(215, 351)


In [213]:
bow_idf_new_order[0].shape

(351,)

In [214]:
pyemd.emd(bow_idf_new_order[0],bow_idf_new_order[50],distance_matrix)

1.4038303975226611

In [215]:
def get_emd_matrix (idf_matrix, distance_matrix):
    emd_matrix = np.zeros((idf_matrix.shape[0],idf_matrix.shape[0]))
    for doc_i in range(idf_matrix.shape[0]):
        for doc_j in range(idf_matrix.shape[0]):
            if doc_j>doc_i:
                emd_matrix[doc_i,doc_j] = pyemd.emd(idf_matrix[doc_i], idf_matrix[doc_j],distance_matrix)
            emd_matrix[doc_j,doc_i] = emd_matrix[doc_i,doc_j]
    return emd_matrix

In [216]:
emd_matrix = get_emd_matrix(bow_idf_new_order,distance_matrix)

In [124]:
np.unique(emd_matrix)

array([0.])

In [217]:
emd_matrix.shape

(215, 215)

In [218]:
#save emd_distance matrix
with open("/Users/mehdiregina/FilRouge/Mehdi/test/emd_matrix_15_70", "wb") as f:
    pickle.dump(emd_matrix,f,protocol=pickle.HIGHEST_PROTOCOL)

In [102]:
#load emd_distance matrix
with open("/Users/mehdiregina/FilRouge/Mehdi/test/emd_matrix_10_80_new", "rb") as f:
    emd_recover = pickle.load(f)

In [163]:
emd_recover.shape

(215, 215)

#### Test Spectral Clustering based on emd distance

In [164]:
#need to transform distance matrix into a similarity matrix
beta = 1
similarity_emd = np.exp(-(beta/emd_recover.std())*emd_recover)

Exponentiel de - distance, permet d'éteindre les distances trop lointaines et ainsi se focaliser sur les distances "locales". Le coefficient beta (hyperparametre) permet d'éteindre plus ou moins rapidement les grandes distances.

In [165]:
similarity_emd[0]

array([1.        , 0.09726392, 0.26499716, 0.02455125, 0.24340178,
       0.47357255, 0.1486064 , 0.08544382, 0.09763911, 0.32121326,
       0.07411192, 0.31197339, 0.59088683, 0.29664255, 0.29580974,
       0.54830148, 0.38641749, 0.25916363, 0.24908991, 0.66192186,
       0.43334991, 0.23190359, 0.59511564, 0.02127318, 0.4983802 ,
       0.26913091, 0.30536242, 0.2104376 , 0.48700336, 0.12989706,
       0.06608328, 0.47126757, 0.44059259, 0.33472661, 0.20189767,
       0.0690035 , 0.64862801, 0.39391953, 0.45891084, 0.52691384,
       0.06910987, 0.4501924 , 0.53330699, 0.11473279, 0.5015904 ,
       0.10557315, 0.10396784, 0.45145695, 0.15457613, 0.44535576,
       0.1371164 , 0.19749538, 0.24652971, 0.14849012, 0.4092673 ,
       0.42131127, 0.35920756, 0.20150504, 0.1589939 , 0.34303822,
       0.21022893, 0.2801016 , 0.53475912, 0.53843338, 0.10142682,
       0.32149469, 0.47469012, 0.09134058, 0.29208209, 0.29208209,
       0.02335227, 0.55978377, 0.60317775, 0.47596546, 0.66813

In [189]:
spectral_c = SpectralClustering(n_clusters=6,affinity='precomputed',n_neighbors=5)
spectral_c.fit(similarity_emd)

SpectralClustering(affinity='precomputed', assign_labels='kmeans', coef0=1,
          degree=3, eigen_solver=None, eigen_tol=0.0, gamma=1.0,
          kernel_params=None, n_clusters=6, n_init=10, n_jobs=1,
          n_neighbors=5, random_state=None)

In [190]:
spectral_c.affinity_matrix_ == similarity_emd

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [191]:
labels = spectral_c.labels_

In [192]:
pd.DataFrame(labels, columns=["Label"]).groupby(["Label"])["Label"].size()

Label
0    17
1    40
2    44
3    54
4    24
5    36
Name: Label, dtype: int64

In [170]:
dict_rand_word_label = dict()
for label in np.unique(labels):
    idx_label = np.where(labels==label)[0]
    idx_rand = np.random.choice(idx_label,size=(12))
    liste_cluster_word = []
    for idx in idx_rand:
        text = " "
        idx_ordered = np.argsort(bow_idf[idx])[::-1]
        for j in range(0, 5):  # nombre de motss
            text += vocab_liste_idf[idx_ordered[j]]+" "
        liste_cluster_word.append(text)
    dict_rand_word_label["cluster"+str(label)] = liste_cluster_word

In [198]:
dict_rand_word_label["cluster5"]

[' version intelligence business tableau reseau ',
 ' qualification conception realisation c application ',
 ' talend creation consultante flux data ',
 ' qualification conception realisation c application ',
 ' sas data scientist indicateurs direction ',
 ' sap solution manager fonctionnelle ibm ',
 ' reports interface parametrage installation oeuvre ',
 ' learning r scientist statistiques marketing ',
 ' sas data r statistique analytics ',
 ' sas data scientist direction amoa ',
 ' learning r scientist statistiques marketing ',
 ' xi migration univers ressources securite ']

#### Test Agglomerative clustering

In [203]:
agg_c = AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='complete')
agg_c.fit(similarity_emd)

AgglomerativeClustering(affinity='precomputed', compute_full_tree='auto',
            connectivity=None, linkage='complete',
            memory=Memory(cachedir=None), n_clusters=5,
            pooling_func=<function mean at 0x110ec2840>)

In [204]:
labels = agg_c.labels_

In [205]:
pd.DataFrame(labels, columns=["Label"]).groupby(["Label"])["Label"].size()

Label
0    75
1    32
2    64
3    23
4    21
Name: Label, dtype: int64

In [152]:
dict_rand_word_label = dict()
for label in np.unique(labels):
    idx_label = np.where(labels==label)[0]
    idx_rand = np.random.choice(idx_label,size=(20))
    liste_cluster_word = []
    for idx in idx_rand:
        text = " "
        idx_ordered = np.argsort(bow_idf[idx])[::-1]
        for j in range(0, 4):  # nombre de motss
            text += vocab_liste_idf[idx_ordered[j]]+" "
        liste_cluster_word.append(text)
    dict_rand_word_label["cluster"+str(label)] = liste_cluster_word

In [209]:
dict_rand_word_label["cluster4"]

[' cognos client ibm bi entrepot ',
 ' edf correction ressources developer application ',
 ' edf correction ressources developer application ',
 ' data science spark r scientist ',
 ' reportings statistiques data etudes spark ',
 ' edf correction ressources developer application ',
 ' projets comptabilite controle microsoft groupe ',
 ' projets comptabilite controle microsoft groupe ',
 ' planning budgetaire optimisation elaboration reporting ',
 ' comptes nt execution sante operations ',
 ' reportings statistiques data etudes spark ',
 ' edf correction ressources developer application ']

#### test dbscan

In [175]:
dbs_c = DBSCAN(eps=2.5,min_samples=3,metric='precomputed')
dbs_c.fit(emd_recover)

DBSCAN(algorithm='auto', eps=2.5, leaf_size=30, metric='precomputed',
    min_samples=3, n_jobs=1, p=None)

In [176]:
pd.DataFrame(dbs_c.labels_, columns=["Label"]).groupby(["Label"])["Label"].size()

Label
-1     150
 0       4
 1       4
 2       4
 3       3
 4       5
 5       3
 6       3
 7       3
 8       4
 9       4
 10      5
 11      7
 12      3
 13      4
 14      3
 15      3
 16      3
Name: Label, dtype: int64